In [25]:
import numpy as np
from scipy.special import factorial
from scipy.optimize import bisect
from scipy.stats import norm

# Poisson Error

Let $\lambda_u$ and $\lambda_l$ be the upper limit and lower limit of confidence level $CL$ for the observed number counts $N$. Then, based on Poisson statistics, these parameters are defined by

\begin{align*}
    \sum_{x = 0}^{N} \frac{\lambda_u^x}{x!} e^{-\lambda_u} &= 1 - CL \\
    \sum_{x = 0}^{N-1} \frac{\lambda_l^x}{x!} e^{-\lambda_l} &= CL, (N \neq 0)
\end{align*}

The lower limit for $N = 0$ is $\lambda_l = 0$. The value for CL are chosen to be the $1\sigma$ limit for Gaussian statistics i.e. $0.8413$.

Reference: [Gehrels, N. 1986, ApJ, 303, 336](https://ui.adsabs.harvard.edu/abs/1986ApJ...303..336G/abstract)

In [67]:
def poisson_limits(N,CL=None,sigma=None):
    # initialize poisson functions
    def poisson(l,x):
        return np.power(l,x)*np.exp(-l)/factorial(x)
    def upper_gehrels(l,N,CL):
        x = np.arange(N+1)
        p = poisson(l,x)
        return np.sum(p) - (1 - CL)
    def lower_gehrels(l,N,CL):
        x = np.arange(N)
        p = poisson(l,x)
        return np.sum(p) - CL
    
    # initialize CL and sigma
    if sigma:
        CL = norm.cdf(sigma)
    if CL:
        sigma = norm.ppf(CL)
    
    if N <= 100: # bisection on exact functions
        upper = bisect(upper_gehrels,0,N*10 + 10,args=(N,CL))
        lower = bisect(lower_gehrels,0,N*10 + 10,args=(N,CL)) if N > 0 else 0
    else: # gaussian approximation
        upper = sigma*np.sqrt(N) + N
        lower = N - sigma*np.sqrt(N)
    return lower, upper

In [68]:
for i in [0,1,5,10,20,50,100,200,500,1000]:
    print(f'The Poisson limits for N = {i} are:',poisson_limits(i,CL=0.9995))

The Poisson limits for N = 0 are: (0, 7.600902459543022)
The Poisson limits for N = 1 are: (0.000500125041753563, 9.99867749762302)
The Poisson limits for N = 5 are: (0.6324910403310469, 17.41063731823772)
The Poisson limits for N = 10 are: (2.699033106031976, 25.255559379265122)
The Poisson limits for N = 20 are: (8.453108207561613, 39.409829428488834)
The Poisson limits for N = 50 are: (29.94782899328154, 77.81469073108656)
The Poisson limits for N = 100 are: (70.33022515951772, 137.3752937205759)
The Poisson limits for N = 200 are: (153.46492468972906, 246.53507531027094)
The Poisson limits for N = 500 are: (426.42158546603855, 573.5784145339614)
The Poisson limits for N = 1000 are: (895.9444082681621, 1104.055591731838)
